<h3>Imports</h3>

In [23]:

from mido import MidiFile, MidiTrack, Message
from mido.midifiles import MetaMessage
from os import listdir
import os
import numpy as np
import pandas as pd

<h3>CONSTANTS</h3>

In [24]:
NR_TRAINING = 1
COMPRESSION_FACTOR = 1
CH_KEY = 'G'
NUMBER_OCTAVES = 3

In [25]:
def changeKey(key, ch_key):
    print('changeKey')
    dict = {'C':48, 'C#':49,'Db':49, 'D':50, 'D#':51,'Eb':51, 'E':52, 'F': 53, 'F#':54,'Gb':54, 'G':55, 'G#':56,'Ab':56, 'A':57, 'A#':58,'Bb':58,'B':59}
    for dict_key, value in dict.items():
        dict[dict_key] = value - 24
    if key!= ch_key:
        transposition = abs(dict[key]-dict[ch_key])
        #print(transposition)
    else:
        transposition = 0
    
    return transposition, dict[ch_key]


In [26]:
def getKey(mid):
    for track in mid.tracks:
        for message in track:
            if message.type == 'key_signature':
                return message.key

In [27]:
def getNoteStartLength(getNoteCurrentOnOf):
    print(getNoteStartLength)
    note_on_length_array = []
    first_time = False
    aux = 0
    b = [x[1] for x in getNoteCurrentOnOf]
    aux = b[0]
    if aux != 0:
        first_time = True

    for i, message in enumerate(getNoteCurrentOnOf):
        if message[2] == 1: 
            start_time = message[1]
            for event in getNoteCurrentOnOf[i:]: 
                if event[0] == message[0] and event[2] == 0:
                    length = event[1] - start_time
                    break
                
            note_on_length_array.append([message[0], start_time, length])

    if first_time == True:
        i = 0
        for note,timer,length in note_on_length_array:
            timer = timer - aux
            note_on_length_array[i]=[note, timer, length]
            i = i+1
    return note_on_length_array

In [28]:
def getTicks(files_dir,nr_training, comp_factor=1, type='Piano left'):
    print('Getting Ticks')
    ticks = []
    counter = 0
    #for file_dir in files_dir:
    for file_path in listdir(files_dir):
        file_path = os.path.join(files_dir,file_path)
        if counter<nr_training:
            print(file_path)
            counter = counter+1
            mid = MidiFile(file_path)                  
            for track in mid.tracks:
                if(track.name==type):
                    num_ticks = 0          
                    for message in track:
                        if not isinstance(message, MetaMessage):
                           if(message.type=='note_on'):
                                num_ticks += int(message.time/comp_factor)
                        ticks.append(num_ticks)
    #print(int(sum(ticks)/len(ticks)))
    #return int(sum(ticks)/len(ticks))
    return max(ticks)

In [29]:
midi_file = 'D:\\Programming\\Musicron_LSTM\\Songs'
mel_train_files = midi_file

nr_training = NR_TRAINING
compression_factor = COMPRESSION_FACTOR
ch_key = CH_KEY
number_octaves = NUMBER_OCTAVES

In [30]:
mel_ticks = getTicks(midi_file, 1, compression_factor, type='Piano left')

Getting Ticks
D:\Programming\Musicron_LSTM\Songs\chp_op18.mid


In [31]:
def getNoteCurrentOnOf(mid, comp_factor, transpose, type):
    print('getNoteCurrentOnOf')
    getNoteCurrentOnOf = []
    velocity =[]
    for track in mid.tracks:
        if(track.name==type):
            current_time = 0
            for message in track:
                    if not isinstance(message, MetaMessage):
                        current_time += int(message.time/comp_factor)
                        bol = False
                        if (message.type == 'note_on'):
                            if(message.velocity!=0):
                                note_onoff = 1
                                bol = True
                            else:
                                note_onoff = 0
                                bol = True
                        if (message.type == 'note_off'):
                            note_onoff = 0
                            bol = True
                        if (message.type == 'velocity'):
                            print("here")
                            
                        if(bol==True):
                            getNoteCurrentOnOf.append([message.note+(12-transpose), current_time, note_onoff])
                            velocity.append(message.velocity)
      
    return getNoteCurrentOnOf

In [41]:
def createPianoRoll(files_dir, ticks, nr_training, comp_factor, ch_key, number_octaves, type):
    print('Creating PianoRoll Matrix')
    number_notes = number_octaves*12
    piano_roll = np.zeros((nr_training,ticks, number_notes))
    
    piano_roll_velocity = np.zeros((nr_training,ticks, number_notes))
    counter = 0
    for i, file in enumerate(listdir(files_dir)):
        if counter<nr_training:
            counter = counter + 1
            file_path = os.path.join(files_dir,file)
            mid = MidiFile(file_path)
            print(file_path)
            key = getKey(mid)
            print(key)
            if key != ch_key:
                print('Transposing %s to %s' % (key, ch_key))
                transpose, low_note = changeKey(key, ch_key)
                high_note = low_note + number_octaves*12
            else:
                print('Same key')
                transpose, low_note = changeKey(key, ch_key)
                high_note = low_note + number_octaves*12
        
            note_time_onoff = getNoteCurrentOnOf(mid, comp_factor, transpose, type)
            note_on_length = getNoteStartLength(note_time_onoff)
            for message in note_on_length:
                if(message[0]<low_note):
                    while(message[0]<low_note):
                        message[0] = message[0] + 12
                    piano_roll[i,message[1]:(message[1]+int(message[2]/2)), message[0]-low_note] = 1
                elif(message[0]>=high_note):
                    while(message[0]>=high_note):
                        message[0] = message[0] - 12
                    piano_roll[i,message[1]:(message[1]+int(message[2]/2)),message[0]-low_note] = 1
                else:
                    piano_roll[i,message[1]:(message[1]+int(message[2]/2)), message[0]-low_note] =  1
          
    return piano_roll, low_note
mel_roll, low_note = createPianoRoll(mel_train_files, mel_ticks, nr_training, compression_factor,ch_key, number_octaves, 'Piano left')

Creating PianoRoll Matrix
D:\Programming\Musicron_LSTM\Songs\chp_op18.mid
Eb
Transposing Eb to G
changeKey
getNoteCurrentOnOf
<function getNoteStartLength at 0x0000017DF4CC3A60>


In [32]:
df = pd.DataFrame(mel_roll[0,:,:])

In [33]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
df.to_csv("test_input%d.csv" %1 , sep=' ',index=False)

In [35]:
def NetOutToPianoRoll(network_output, threshold):
    piano_roll = []
    for i, timestep in enumerate(network_output):
        if np.amax(timestep) > threshold:
            pos = 0
            pos = np.argmax(timestep)
            timestep[:] = np.zeros(timestep.shape)
            timestep[pos] = 1
        else:
            timestep[:] = np.zeros(timestep.shape)
        piano_roll.append(timestep)
       
    return np.array(piano_roll)

In [36]:
net_roll = NetOutToPianoRoll(mel_roll, threshold=0.4)

In [37]:
def createMidi(n,piano_roll, directory, mel_test_file, threshold, rez_factor, tempo_new,low_note):
    print(createMidi)
  

    ticks_per_beat = int(480)
    mid = MidiFile(type=1, ticks_per_beat=ticks_per_beat)

    track = MidiTrack()
    
    mid.tracks.append(track)

    delta_times = [0]
    time_list = []
    track.append(MetaMessage('set_tempo', tempo=tempo_new))
    for k in range(piano_roll.shape[1]):#initial starting values
        if piano_roll[0, k].any() == 1:
            track.append(Message('note_on', note=k+low_note, velocity=100, time=0))
            delta_times.append(0)
    counter = 0  
    ok = False
    first_note = True
    for j in range(piano_roll.shape[0]-1):#all values between first and last one
        ok = False
        for k in range(piano_roll.shape[1]):
            if (piano_roll[j+1, k] == 0 and piano_roll[j, k] == 1):
                if first_note == True:
                    time = (j+1)*rez_factor - sum(delta_times)
                else:
                    time = (j-counter+1)*rez_factor - sum(delta_times)
                time_list.append(time)        
                delta_times.append(time)
                #counter = counter + 1
                ok = True
                if piano_roll[j+1, k] == 1 and piano_roll[j, k] == 0:
                    track.append(Message('note_on', note=k+low_note, velocity=100, time=time))
                if piano_roll[j+1, k] == 0 and piano_roll[j, k] == 1:
                    track.append(Message('note_off', note=k+low_note, velocity=0, time=time))
                counter=counter+1
                    
                    
            if (piano_roll[j+1, k] == 1 and piano_roll[j, k] == 0):
                if not time_list:
                    time = 0
                else:
                    time = time_list.pop()
                         
                if piano_roll[j+1, k] == 1 and piano_roll[j, k] == 0:
                    #track.append(Message('note_on', note=k+low_note, velocity=100, time=time))
                    # Added 2022
                    # Want to maintain legato
                    track.append(Message('note_on', note=k+low_note, velocity=100, time=0))
                if piano_roll[j+1, k] == 0 and piano_roll[j, k] == 1:
                    track.append(Message('note_off', note=k+low_note, velocity=0, time=time))
                ok = True
                first_note=False
            if ok == True:
                break
                             
    mid.save(mel_test_file)

In [38]:
def timeSignature(files_dir, compression_factor, numerator, denominator):
     print(timeSignature)
     num_files = len(files_dir)
     time_sig = np.zeros((num_files), dtype=np.int)
     nr_files = 0
     mean = 0
     ct = 0
     
     for i,file_name in enumerate(listdir(files_dir)):
        file_path = os.path.join(files_dir,file_name)
        mid = MidiFile(file_path)
        tpb = mid.ticks_per_beat                  
        for track in mid.tracks:
            for message in track:
                if(message.type == 'time_signature'):
                    if(message.numerator == numerator and message.denominator == denominator):
                        print('%s         ' % (file_name),end='')
                        print('%d %d %d' % (message.numerator, message.denominator, mid.ticks_per_beat))
                        tpb = mid.ticks_per_beat
                        if message.denominator == 8:
                            tpb = mid.ticks_per_beat/2
                        if message.denominator == 16:
                            tpb = mid.ticks_per_beat/4
                        fac = tpb * message.numerator
                        fac = int(fac/compression_factor)
                        time_sig=fac
                        nr_files = nr_files+1
                if message.type == 'set_tempo':
                    mean = mean+message.tempo
                    ct += 1
     tempo = int(mean/ct)
     return time_sig, tpb, tempo  

seq_length, tpb, tempo  = timeSignature(midi_file,compression_factor,3,8)

<function timeSignature at 0x0000017DE3F92820>


C:\Users\bogda\AppData\Local\Temp/ipykernel_8500/3402539254.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  time_sig = np.zeros((num_files), dtype=np.int)


chp_op18.mid         3 8 480


In [39]:
createMidi(tpb,mel_roll[0], r'D:\Programming\Musicron_LSTM\Songs', 'TesterV2.mid', 0.1,1, tempo, low_note) 


<function createMidi at 0x0000017DE3D25EE0>
